In [1]:
import cv2
import numpy as np

In [2]:
def double_thresholding(src):
    dst = src.copy()

    # dst => 0 ~ 255
    dst -= dst.min()
    dst /= dst.max()
    dst *= 255
    dst = dst.astype(np.uint8)
    (h, w) = dst.shape

    high_threshold_value, _ = cv2.threshold(dst, 0, 255, cv2.THRESH_OTSU)

    print('highthreshold')
    print(high_threshold_value)
    low_threshold_value = high_threshold_value * 0.4

    for row in range(h):
        for col in range(w):
            if dst[row, col] >= high_threshold_value:
                dst[row, col] = 255
            elif dst[row, col] < low_threshold_value:
                dst[row, col] = 0
            else:
                #######################################
                # TODO                                #
                # High 값 보다 작고 Low 값 보다 큰 경우   #
                #######################################
                dst[row, col] = 128
    
    dx = [-1, 0, 1, -1, 0, 1, -1, 0, 1]
    dy = [-1, -1, -1, 0, 0, 0, 1, 1, 1]

    for row in range(h):
        for col in range(w):
            if dst[row, col] == 255 or dst[row, col] == 0:
                continue

            s = [(row, col)]
            visited = []
            isStrong = False

            while len(s) > 0:
                sh, sw = s.pop()
                visited.append((sh, sw))

                for i in range(9):
                    if sw+dx[i] < 0 or sw+dx[i] >= w or sh+dy[i] < 0 or sh+dy[i] >= h:
                        continue

                    if dst[sh+dy[i], sw+dx[i]] == 255:
                        isStrong = True
                    elif dst[sh+dy[i], sw+dx[i]] == 128:
                        s.append((sh+dy[i], sw+dx[i]))
            
            for sh, sw in visited:
                dst[sh, sw] = isStrong * 255

    return dst

In [3]:
img = cv2.imread("Lena.png", cv2.IMREAD_GRAYSCALE)
dst = double_thresholding(img)
cv2.imshow("dst", dst)
cv2.waitKey()
cv2.destroyAllWindows()

UFuncTypeError: Cannot cast ufunc 'divide' output from dtype('float64') to dtype('uint8') with casting rule 'same_kind'